In [183]:
!pip install requests

In [184]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import requests
import json

def checkindicator(url):
    r= requests.get(url)
    r = r.json()
    periods = r['series']['docs'][0]['period']
    values = r['series']['docs'][0]['value']
    dataset = r['series']['docs'][0]['dataset_name']
    indicators = pd.DataFrame(values,index=periods)
    indicators.columns = [dataset]
    return indicators

unemployment_rate = checkindicator("https://api.db.nomics.world/v22/series/BLS/ln/LNS13327707?observations=1")
PPI = checkindicator("https://api.db.nomics.world/v22/series/OECD/DP_LIVE/USA.PPI.TOT_MKT.IDX2015.M?observations=1")
CPI = checkindicator("https://api.db.nomics.world/v22/series/IMF/CPI/M.US.PCPI_IX?observations=1")
interest_rate = checkindicator("https://api.db.nomics.world/v22/series/OECD/MEI/USA.IRSTCI01.ST.M?observations=1")


In [185]:
# unemployment rate
# "https://db.nomics.world/BLS/ln/LNS13327707"
# unemployment_rate = checkindicator("https://api.db.nomics.world/v22/series/BLS/ln/LNS13327707?observations=1")

new_unemployment_rate = unemployment_rate.loc[unemployment_rate.index>='1999-01']
print(new_unemployment_rate)


         Labor Force Statistics including the National Unemployment Rate
1999-01                                                4.5              
1999-02                                                4.6              
1999-03                                                4.4              
1999-04                                                4.5              
1999-05                                                4.4              
...                                                    ...              
2023-05                                                3.9              
2023-06                                                3.7              
2023-07                                                3.7              
2023-08                                                4.0              
2023-09                                                4.0              

[297 rows x 1 columns]


In [186]:
# PPI with 2015 = 100
# https://db.nomics.world/OECD/DP_LIVE/USA.PPI.TOT_MKT.IDX2015.M
# PPI = checkindicator("https://api.db.nomics.world/v22/series/OECD/DP_LIVE/USA.PPI.TOT_MKT.IDX2015.M?observations=1")

new_PPI = PPI.loc[PPI.index>='1999-01']
print(new_PPI)
print(new_PPI.columns)


         OECD Data Live dataset
1999-01                68.01402
1999-02                67.85233
1999-03                68.06791
1999-04                68.66074
1999-05                68.82242
...                         ...
2022-08               135.53970
2022-09               135.82860
2022-10               136.88980
2022-11               136.27220
2022-12               131.74990

[288 rows x 1 columns]
Index(['OECD Data Live dataset'], dtype='object')


In [187]:
# CPI
# https://db.nomics.world/IMF/CPI/M.US.PCPI_IX
# CPI = checkindicator("https://api.db.nomics.world/v22/series/IMF/CPI/M.US.PCPI_IX?observations=1")
new_CPI = CPI.loc[CPI.index>='1999-01']
print(new_CPI)

         Consumer Price Index (CPI)
1999-01                   75.347790
1999-02                   75.439510
1999-03                   75.668809
1999-04                   76.219128
1999-05                   76.219128
...                             ...
2023-04                  139.121921
2023-05                  139.472290
2023-06                  139.922634
2023-07                  140.189539
2023-08                  140.801768

[296 rows x 1 columns]


In [188]:
#interest rate
# https://db.nomics.world/OECD/MEI/USA.IRSTCI01.ST.M
interest_rate = checkindicator("https://api.db.nomics.world/v22/series/OECD/MEI/USA.IRSTCI01.ST.M?observations=1")
new_interest_rate = interest_rate.loc[interest_rate.index>='1999-01']
print(new_interest_rate)

         Main Economic Indicators Publication
1999-01                                  4.63
1999-02                                  4.76
1999-03                                  4.81
1999-04                                  4.74
1999-05                                  4.74
...                                       ...
2023-05                                  5.06
2023-06                                  5.08
2023-07                                  5.12
2023-08                                  5.33
2023-09                                  5.33

[297 rows x 1 columns]


In [189]:
#normalized GDP  this is the best I can get so far
# https://db.nomics.world/OECD/MEI_CLI/LORSGPNO.USA.M
normalized_GDP = checkindicator('https://api.db.nomics.world/v22/series/OECD/MEI_CLI/LORSGPNO.USA.M?observations=1')
new_normalized_GDP = normalized_GDP.loc[normalized_GDP.index >= '1999-01']
print(new_normalized_GDP)

         Composite Leading Indicators (MEI)
1999-01                           100.66290
1999-02                           100.68400
1999-03                           100.69920
1999-04                           100.72230
1999-05                           100.76580
...                                     ...
2023-01                            99.86009
2023-02                            99.88584
2023-03                            99.90692
2023-04                            99.92532
2023-05                            99.94305

[293 rows x 1 columns]


In [190]:
# new_CPI['PPI'] = new_PPI['OECD Data Live dataset']

combined_df = new_CPI
combined_df['PPI'] = new_PPI['OECD Data Live dataset']
combined_df['Unemployment_rate'] = new_unemployment_rate['Labor Force Statistics including the National Unemployment Rate']
combined_df['interest_rate'] = new_interest_rate['Main Economic Indicators Publication']
combined_df['normalized_GDP'] = new_normalized_GDP['Composite Leading Indicators (MEI)']


C:\Users\jrjol\AppData\Local\Temp\ipykernel_1980\3796664431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['PPI'] = new_PPI['OECD Data Live dataset']
C:\Users\jrjol\AppData\Local\Temp\ipykernel_1980\3796664431.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Unemployment_rate'] = new_unemployment_rate['Labor Force Statistics including the National Unemployment Rate']
C:\Users\jrjol\AppData\Local\Temp\ipykernel_1980\3796664431.py:6: SettingWithCopyWarning: 
A value is trying 

In [191]:
print(combined_df)
print(len(combined_df))

         Consumer Price Index (CPI)       PPI  Unemployment_rate  \
1999-01                   75.347790  68.01402                4.5   
1999-02                   75.439510  67.85233                4.6   
1999-03                   75.668809  68.06791                4.4   
1999-04                   76.219128  68.66074                4.5   
1999-05                   76.219128  68.82242                4.4   
...                             ...       ...                ...   
2023-04                  139.121921       NaN                3.6   
2023-05                  139.472290       NaN                3.9   
2023-06                  139.922634       NaN                3.7   
2023-07                  140.189539       NaN                3.7   
2023-08                  140.801768       NaN                4.0   

         interest_rate  normalized_GDP  
1999-01           4.63       100.66290  
1999-02           4.76       100.68400  
1999-03           4.81       100.69920  
1999-04           4

In [192]:
spy_data = yf.download("^GSPC", start="1999-01-01", end="2023-09-01", interval="1mo")
'''2023-08-01 (2023-08) represents the Open High Low Close Volume for '''
spy_data = spy_data.reindex(combined_df.index)
print(spy_data.head())
print(len(spy_data))

[*********************100%***********************]  1 of 1 completed
                Open         High          Low        Close    Adj Close  \
1999-01  1229.229980  1280.369995  1205.459961  1279.640015  1279.640015   
1999-02  1279.640015  1283.839966  1211.890015  1238.329956  1238.329956   
1999-03  1238.329956  1323.819946  1216.030029  1286.369995  1286.369995   
1999-04  1286.369995  1371.560059  1282.560059  1335.180054  1335.180054   
1999-05  1335.180054  1375.979980  1277.310059  1301.839966  1301.839966   

              Volume  
1999-01  16213500000  
1999-02  14555860000  
1999-03  18002500000  
1999-04  18523200000  
1999-05  15880260000  
296


In [193]:
spy_data = spy_data.reindex(combined_df.index)
Main_df = pd.concat([combined_df, spy_data], axis=1)


# Print the result
print(Main_df)

         Consumer Price Index (CPI)       PPI  Unemployment_rate  \
1999-01                   75.347790  68.01402                4.5   
1999-02                   75.439510  67.85233                4.6   
1999-03                   75.668809  68.06791                4.4   
1999-04                   76.219128  68.66074                4.5   
1999-05                   76.219128  68.82242                4.4   
...                             ...       ...                ...   
2023-04                  139.121921       NaN                3.6   
2023-05                  139.472290       NaN                3.9   
2023-06                  139.922634       NaN                3.7   
2023-07                  140.189539       NaN                3.7   
2023-08                  140.801768       NaN                4.0   

         interest_rate  normalized_GDP         Open         High          Low  \
1999-01           4.63       100.66290  1229.229980  1280.369995  1205.459961   
1999-02           4.7

In [194]:
feature_scaler=MinMaxScaler()
scaled_data=feature_scaler.fit_transform(Main_df)
min_values = feature_scaler.data_min_
max_values = feature_scaler.data_max_


scaling_factors_df = pd.DataFrame({'Feature': Main_df.columns,
                                   'Min Value': min_values,
                                   'Max Value': max_values})

print(scaling_factors_df)

print(scaled_data)
print(len(scaled_data))


                       Feature     Min Value     Max Value
0   Consumer Price Index (CPI)  7.534779e+01  1.408018e+02
1                          PPI  6.785233e+01  1.414271e+02
2            Unemployment_rate  3.600000e+00  1.510000e+01
3                interest_rate  5.000000e-02  6.540000e+00
4               normalized_GDP  9.203044e+01  1.018704e+02
5                         Open  7.295700e+02  4.778140e+03
6                         High  8.329800e+02  4.818620e+03
7                          Low  6.667900e+02  4.560000e+03
8                        Close  7.350900e+02  4.766180e+03
9                    Adj Close  7.350900e+02  4.766180e+03
10                      Volume  1.455586e+10  1.621854e+11
[[0.         0.00219763 0.07826087 ... 0.13508752 0.13508752 0.01122838]
 [0.00140129 0.         0.08695652 ... 0.12483966 0.12483966 0.        ]
 [0.0049045  0.00293008 0.06956522 ... 0.13675704 0.13675704 0.02334655]
 ...
 [0.98656867        nan 0.00869565 ... 0.92165884 0.92165884 0.49737

In [195]:
look_back=8

def create_dataset(data, look_back):
    X, y = [], []
    for i in range(look_back, len(data)-1):
        X.append(data[i-look_back:i, :])
        y.append(data[i+1, 8])
    return np.array(X), np.array(y)

In [196]:
train_data, test_data=train_test_split(scaled_data, test_size=0.4, shuffle=False)
val_data, test_data = train_test_split(test_data, test_size=0.5, shuffle=False)

X_train, Y_train=create_dataset(train_data, look_back)
X_test, Y_test=create_dataset(test_data, look_back)
X_val, Y_val=create_dataset(val_data, look_back)

In [197]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

optimizer=Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error')

early_stopping=EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

model.fit(X_train, Y_train, epochs=50, batch_size=8, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/50
21/21 [==============================] - 4s 45ms/step - loss: 0.0078 - val_loss: 0.0705
Epoch 2/50
21/21 [==============================] - 0s 9ms/step - loss: 0.0014 - val_loss: 0.0427
Epoch 3/50
21/21 [==============================] - 0s 10ms/step - loss: 8.7866e-04 - val_loss: 0.0260
Epoch 4/50
21/21 [==============================] - 0s 9ms/step - loss: 6.4241e-04 - val_loss: 0.0110
Epoch 5/50
21/21 [==============================] - 0s 9ms/step - loss: 6.3630e-04 - val_loss: 0.0161
Epoch 6/50
21/21 [==============================] - 0s 8ms/step - loss: 5.2639e-04 - val_loss: 0.0123
Epoch 7/50
21/21 [==============================] - 0s 7ms/step - loss: 4.6424e-04 - val_loss: 0.0097
Epoch 8/50
21/21 [==============================] - 0s 9ms/step - loss: 4.2485e-04 - val_loss: 0.0112
Epoch 9/50
21/21 [==============================] - 0s 9ms/step - loss: 4.7024e-04 - val_loss: 0.0128
Epoch 10/50
21/21 [==============================] - 0s 9ms/step - loss: 6.0426e-04 - va

In [198]:
predicted_prices_scaled=model.predict(X_test)

2/2 [==============================] - 0s 4ms/step
